Notebook for the one-hot autoencoder

In [1]:
from communications import *
from model import *
import numpy as np
import matplotlib.pyplot as plt
import os.path

%load_ext autoreload
%autoreload 2

In [2]:
import pickle

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [3]:
#X_train,y_train=np.eye(256),np.eye(256)
X_train,y_train=createBitVectors(),createBitVectors()

In [4]:
SNR=1
add_name='_1'

In [ ]:
autoencoderAWGN,encoderAWGN,decoderAWGN=OHAutoencoder(5*256,5*256,AWGN,SNR,use_BN=False,use_LN=False)

In [ ]:
boundaries = [2**16,2**17]
values = [1e-4,1e-5,1e-6]
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
autoencoderAWGN,encoderAWGN,decoderAWGN=OHAutoencoder2(5*256,5*256,AWGN,SNR,use_BN=True,use_LN=False,lr=1e-4,encoder_activation='tanh',hidden_activation='relu',optim=Adam,lookahead=False,gradient_centralization=False)

In [ ]:
autoencoderAWGN,encoderAWGN,decoderAWGN=OHAutoencoder(5*256,5*256,multi_AWGN,use_BN=False,use_LN=False)

In [ ]:
from tensorflow.keras.models import load_model

encoderAWGN,decoderAWGN=load_model('AWGNEncoder_3_test.h5',custom_objects={'AWGN':AWGN}),load_model('AWGNDecoder_3_test.h5',custom_objects={'AWGN':AWGN})
autoencoderAWGN,encoderAWGN,decoderAWGN=recompile(encoderAWGN,decoderAWGN,AWGN,noise_param=SNR,optim=Adam,lookahead=True,gradient_centralization=True,lr=1e-6)

In [5]:
def schedule(epoch,lr):
    if epoch<3*2**16:
        return 1e-3
    else:
        return 1e-5
    

scheduler=tf.keras.callbacks.LearningRateScheduler(schedule)


In [6]:
boundaries = [2**16,2**17]
values = [1e-4,1e-5,1e-6]
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
autoencoderAWGN,encoderAWGN,decoderAWGN=Autoencoder('normal','normal',[32,64,128],[128,64,32],AWGN,SNR,use_BN=True,use_LN=False,lr=1e-4,encoder_activation='tanh',hidden_activation=mish,optim=RectifiedAdam,lookahead=True,gradient_centralization=True)

In [ ]:
checkpoint=tf.keras.callbacks.ModelCheckpoint('AWGNAutoencoderCheckpoint'+add_name+'.h5',monitor='loss',save_best_only=True)
history=autoencoderAWGN.fit(x=X_train,y=y_train,batch_size=256,epochs=2**18,callbacks=[checkpoint,scheduler],verbose=2)
save_obj(history.history,'history'+add_name)
autoencoderAWGN.save('AWGNAutoencoder'+add_name+'.h5')
encoderAWGN.save('AWGNEncoder'+add_name+'.h5')
decoderAWGN.save('AWGNDecoder'+add_name+'.h5')

Epoch 1/262144
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
1/1 - 0s - loss: 0.8732 - ber_metric: 0.5117 - acc: 0.1758 - lr: 0.0010
Epoch 2/262144
1/1 - 0s - loss: 0.8469 - ber_metric: 0.5010 - acc: 0.1602 - lr: 0.0010
Epoch 3/262144
1/1 - 0s - loss: 0.8616 - ber_metric: 0.5107 - acc: 0.1641 - lr: 0.0010
Epoch 4/262144
1/1 - 0s - loss: 0.8617 - ber_metric: 0.5088 - acc: 0.1680 - lr: 0.0010
Epoch 5/262144
1/1 - 0s - loss: 0.8736 - ber_metric: 0.5127 - acc: 0.1641 - lr: 0.0010
Epoch 6/262144
1/1 - 0s - loss: 0.8644 - ber_metric: 0.5044 - acc: 0.1602 - lr: 0.0010
Epoch 7/262144
1/1 - 0s - loss: 0.8558 - ber_metric: 0.5039 - acc: 0.1641 - lr: 0.0010
Epoch 8/262144
1/1 - 0s - loss: 0.8520 - ber_metric: 0.5093 - acc: 0.1797 - lr: 0.0010
Epoch 9/262144
1/1 - 0s - loss: 0.8541 - ber_metric: 0.5039 - acc: 0.1445 - lr: 0.0010
Epoch 10/262144
1/1 - 0s - loss: 0.8514 - ber_metric: 0.5107 - acc: 0.1523 - lr: 0.0010
Epoch 11/262144
1/1 - 0s - loss: 0.8397 - ber_me

1/1 - 0s - loss: 0.5374 - ber_metric: 0.2739 - acc: 0.2148 - lr: 0.0010
Epoch 91/262144
1/1 - 0s - loss: 0.5454 - ber_metric: 0.2847 - acc: 0.2422 - lr: 0.0010
Epoch 92/262144
1/1 - 0s - loss: 0.5391 - ber_metric: 0.2622 - acc: 0.2031 - lr: 0.0010
Epoch 93/262144
1/1 - 0s - loss: 0.5328 - ber_metric: 0.2759 - acc: 0.2656 - lr: 0.0010
Epoch 94/262144
1/1 - 0s - loss: 0.5158 - ber_metric: 0.2549 - acc: 0.2188 - lr: 0.0010
Epoch 95/262144
1/1 - 0s - loss: 0.5207 - ber_metric: 0.2603 - acc: 0.2422 - lr: 0.0010
Epoch 96/262144
1/1 - 0s - loss: 0.5234 - ber_metric: 0.2622 - acc: 0.2188 - lr: 0.0010
Epoch 97/262144
1/1 - 0s - loss: 0.5326 - ber_metric: 0.2749 - acc: 0.2383 - lr: 0.0010
Epoch 98/262144
1/1 - 0s - loss: 0.5336 - ber_metric: 0.2754 - acc: 0.2188 - lr: 0.0010
Epoch 99/262144
1/1 - 0s - loss: 0.5259 - ber_metric: 0.2515 - acc: 0.2617 - lr: 0.0010
Epoch 100/262144
1/1 - 0s - loss: 0.5139 - ber_metric: 0.2539 - acc: 0.2656 - lr: 0.0010
Epoch 101/262144
1/1 - 0s - loss: 0.5020 - ber_

Epoch 183/262144
1/1 - 0s - loss: 0.3893 - ber_metric: 0.1582 - acc: 0.2422 - lr: 0.0010
Epoch 184/262144
1/1 - 0s - loss: 0.3875 - ber_metric: 0.1621 - acc: 0.2773 - lr: 0.0010
Epoch 185/262144
1/1 - 0s - loss: 0.3921 - ber_metric: 0.1572 - acc: 0.2734 - lr: 0.0010
Epoch 186/262144
1/1 - 0s - loss: 0.3894 - ber_metric: 0.1641 - acc: 0.2539 - lr: 0.0010
Epoch 187/262144
1/1 - 0s - loss: 0.3938 - ber_metric: 0.1636 - acc: 0.2305 - lr: 0.0010
Epoch 188/262144
1/1 - 0s - loss: 0.3949 - ber_metric: 0.1592 - acc: 0.2266 - lr: 0.0010
Epoch 189/262144
1/1 - 0s - loss: 0.3849 - ber_metric: 0.1562 - acc: 0.2617 - lr: 0.0010
Epoch 190/262144
1/1 - 0s - loss: 0.3953 - ber_metric: 0.1675 - acc: 0.2539 - lr: 0.0010
Epoch 191/262144
1/1 - 0s - loss: 0.3771 - ber_metric: 0.1499 - acc: 0.2656 - lr: 0.0010
Epoch 192/262144
1/1 - 0s - loss: 0.3787 - ber_metric: 0.1548 - acc: 0.3008 - lr: 0.0010
Epoch 193/262144
1/1 - 0s - loss: 0.3876 - ber_metric: 0.1616 - acc: 0.2461 - lr: 0.0010
Epoch 194/262144
1/1 

Epoch 276/262144
1/1 - 0s - loss: 0.3054 - ber_metric: 0.0947 - acc: 0.3008 - lr: 0.0010
Epoch 277/262144
1/1 - 0s - loss: 0.3231 - ber_metric: 0.1133 - acc: 0.3125 - lr: 0.0010
Epoch 278/262144
1/1 - 0s - loss: 0.3184 - ber_metric: 0.1113 - acc: 0.3008 - lr: 0.0010
Epoch 279/262144
1/1 - 0s - loss: 0.3153 - ber_metric: 0.1162 - acc: 0.3008 - lr: 0.0010
Epoch 280/262144
1/1 - 0s - loss: 0.3059 - ber_metric: 0.1045 - acc: 0.3125 - lr: 0.0010
Epoch 281/262144
1/1 - 0s - loss: 0.3170 - ber_metric: 0.1099 - acc: 0.2891 - lr: 0.0010
Epoch 282/262144
1/1 - 0s - loss: 0.3058 - ber_metric: 0.1021 - acc: 0.2969 - lr: 0.0010
Epoch 283/262144
1/1 - 0s - loss: 0.3170 - ber_metric: 0.1162 - acc: 0.2930 - lr: 0.0010
Epoch 284/262144
1/1 - 0s - loss: 0.3053 - ber_metric: 0.1006 - acc: 0.2852 - lr: 0.0010
Epoch 285/262144
1/1 - 0s - loss: 0.3079 - ber_metric: 0.1104 - acc: 0.3008 - lr: 0.0010
Epoch 286/262144
1/1 - 0s - loss: 0.3215 - ber_metric: 0.1167 - acc: 0.2852 - lr: 0.0010
Epoch 287/262144
1/1 

1/1 - 0s - loss: 0.2619 - ber_metric: 0.0820 - acc: 0.3281 - lr: 0.0010
Epoch 369/262144
1/1 - 0s - loss: 0.2648 - ber_metric: 0.0884 - acc: 0.3203 - lr: 0.0010
Epoch 370/262144
1/1 - 0s - loss: 0.2659 - ber_metric: 0.0845 - acc: 0.2617 - lr: 0.0010
Epoch 371/262144
1/1 - 0s - loss: 0.2615 - ber_metric: 0.0781 - acc: 0.3359 - lr: 0.0010
Epoch 372/262144
1/1 - 0s - loss: 0.2668 - ber_metric: 0.0864 - acc: 0.2930 - lr: 0.0010
Epoch 373/262144
1/1 - 0s - loss: 0.2617 - ber_metric: 0.0762 - acc: 0.2773 - lr: 0.0010
Epoch 374/262144
1/1 - 0s - loss: 0.2606 - ber_metric: 0.0825 - acc: 0.3203 - lr: 0.0010
Epoch 375/262144
1/1 - 0s - loss: 0.2718 - ber_metric: 0.0850 - acc: 0.3047 - lr: 0.0010
Epoch 376/262144
1/1 - 0s - loss: 0.2571 - ber_metric: 0.0728 - acc: 0.3008 - lr: 0.0010
Epoch 377/262144
1/1 - 0s - loss: 0.2618 - ber_metric: 0.0786 - acc: 0.3125 - lr: 0.0010
Epoch 378/262144
1/1 - 0s - loss: 0.2486 - ber_metric: 0.0747 - acc: 0.2617 - lr: 0.0010
Epoch 379/262144
1/1 - 0s - loss: 0.25

...

Epoch 25915/262144
1/1 - 0s - loss: 0.0388 - ber_metric: 0.0142 - acc: 0.2852 - lr: 0.0010
Epoch 25916/262144
1/1 - 0s - loss: 0.0250 - ber_metric: 0.0098 - acc: 0.2578 - lr: 0.0010
Epoch 25917/262144
1/1 - 0s - loss: 0.0254 - ber_metric: 0.0068 - acc: 0.2656 - lr: 0.0010
Epoch 25918/262144
1/1 - 0s - loss: 0.0350 - ber_metric: 0.0122 - acc: 0.3047 - lr: 0.0010
Epoch 25919/262144
1/1 - 0s - loss: 0.0409 - ber_metric: 0.0112 - acc: 0.2656 - lr: 0.0010
Epoch 25920/262144
1/1 - 0s - loss: 0.0337 - ber_metric: 0.0117 - acc: 0.2656 - lr: 0.0010
Epoch 25921/262144
1/1 - 0s - loss: 0.0346 - ber_metric: 0.0127 - acc: 0.2578 - lr: 0.0010
Epoch 25922/262144
1/1 - 0s - loss: 0.0440 - ber_metric: 0.0171 - acc: 0.2930 - lr: 0.0010
Epoch 25923/262144
1/1 - 0s - loss: 0.0285 - ber_metric: 0.0107 - acc: 0.2656 - lr: 0.0010
Epoch 25924/262144
1/1 - 0s - loss: 0.0317 - ber_metric: 0.0107 - acc: 0.2695 - lr: 0.0010
Epoch 25925/262144
1/1 - 0s - loss: 0.0358 - ber_metric: 0.0171 - acc: 0.2852 - lr: 0.0010

1/1 - 0s - loss: 0.0315 - ber_metric: 0.0122 - acc: 0.2812 - lr: 0.0010
Epoch 26006/262144
1/1 - 0s - loss: 0.0349 - ber_metric: 0.0117 - acc: 0.2539 - lr: 0.0010
Epoch 26007/262144
1/1 - 0s - loss: 0.0323 - ber_metric: 0.0103 - acc: 0.2422 - lr: 0.0010
Epoch 26008/262144
1/1 - 0s - loss: 0.0332 - ber_metric: 0.0112 - acc: 0.3008 - lr: 0.0010
Epoch 26009/262144
1/1 - 0s - loss: 0.0266 - ber_metric: 0.0103 - acc: 0.3047 - lr: 0.0010
Epoch 26010/262144
1/1 - 0s - loss: 0.0381 - ber_metric: 0.0161 - acc: 0.2656 - lr: 0.0010
Epoch 26011/262144
1/1 - 0s - loss: 0.0406 - ber_metric: 0.0156 - acc: 0.2578 - lr: 0.0010
Epoch 26012/262144
1/1 - 0s - loss: 0.0268 - ber_metric: 0.0098 - acc: 0.2930 - lr: 0.0010
Epoch 26013/262144
1/1 - 0s - loss: 0.0437 - ber_metric: 0.0146 - acc: 0.3047 - lr: 0.0010
Epoch 26014/262144
1/1 - 0s - loss: 0.0276 - ber_metric: 0.0107 - acc: 0.3203 - lr: 0.0010
Epoch 26015/262144
1/1 - 0s - loss: 0.0339 - ber_metric: 0.0117 - acc: 0.2656 - lr: 0.0010
Epoch 26016/262144

Epoch 26096/262144
1/1 - 0s - loss: 0.0272 - ber_metric: 0.0107 - acc: 0.2812 - lr: 0.0010
Epoch 26097/262144
1/1 - 0s - loss: 0.0322 - ber_metric: 0.0137 - acc: 0.2852 - lr: 0.0010
Epoch 26098/262144
1/1 - 0s - loss: 0.0265 - ber_metric: 0.0093 - acc: 0.2812 - lr: 0.0010
Epoch 26099/262144
1/1 - 0s - loss: 0.0294 - ber_metric: 0.0083 - acc: 0.2969 - lr: 0.0010
Epoch 26100/262144
1/1 - 0s - loss: 0.0316 - ber_metric: 0.0132 - acc: 0.2773 - lr: 0.0010
Epoch 26101/262144
1/1 - 0s - loss: 0.0372 - ber_metric: 0.0127 - acc: 0.2656 - lr: 0.0010
Epoch 26102/262144
1/1 - 0s - loss: 0.0347 - ber_metric: 0.0112 - acc: 0.2461 - lr: 0.0010
Epoch 26103/262144
1/1 - 0s - loss: 0.0283 - ber_metric: 0.0117 - acc: 0.2852 - lr: 0.0010
Epoch 26104/262144
1/1 - 0s - loss: 0.0341 - ber_metric: 0.0117 - acc: 0.2578 - lr: 0.0010
Epoch 26105/262144
1/1 - 0s - loss: 0.0258 - ber_metric: 0.0088 - acc: 0.2539 - lr: 0.0010
Epoch 26106/262144
1/1 - 0s - loss: 0.0433 - ber_metric: 0.0151 - acc: 0.2812 - lr: 0.0010

1/1 - 0s - loss: 0.0309 - ber_metric: 0.0132 - acc: 0.2539 - lr: 0.0010
Epoch 26187/262144
1/1 - 0s - loss: 0.0420 - ber_metric: 0.0181 - acc: 0.2617 - lr: 0.0010
Epoch 26188/262144
1/1 - 0s - loss: 0.0327 - ber_metric: 0.0151 - acc: 0.2461 - lr: 0.0010
Epoch 26189/262144
1/1 - 0s - loss: 0.0390 - ber_metric: 0.0132 - acc: 0.2578 - lr: 0.0010
Epoch 26190/262144
1/1 - 0s - loss: 0.0375 - ber_metric: 0.0122 - acc: 0.2695 - lr: 0.0010
Epoch 26191/262144
1/1 - 0s - loss: 0.0410 - ber_metric: 0.0156 - acc: 0.2539 - lr: 0.0010
Epoch 26192/262144
1/1 - 0s - loss: 0.0255 - ber_metric: 0.0093 - acc: 0.2617 - lr: 0.0010
Epoch 26193/262144
1/1 - 0s - loss: 0.0335 - ber_metric: 0.0142 - acc: 0.3047 - lr: 0.0010
Epoch 26194/262144
1/1 - 0s - loss: 0.0378 - ber_metric: 0.0132 - acc: 0.2930 - lr: 0.0010
Epoch 26195/262144
1/1 - 0s - loss: 0.0308 - ber_metric: 0.0098 - acc: 0.2500 - lr: 0.0010
Epoch 26196/262144
1/1 - 0s - loss: 0.0323 - ber_metric: 0.0117 - acc: 0.2773 - lr: 0.0010
Epoch 26197/262144

Epoch 26277/262144
1/1 - 0s - loss: 0.0274 - ber_metric: 0.0088 - acc: 0.3281 - lr: 0.0010
Epoch 26278/262144
1/1 - 0s - loss: 0.0356 - ber_metric: 0.0151 - acc: 0.3125 - lr: 0.0010


In [ ]:
computeVarianceAWGN(encoderAWGN,do_polar=True,print_code=True,round=True)

In [ ]:
testLinearAWGN(encoderAWGN)

In [ ]:
np.count_non_zero(np.round(encoderAWGN.predict(np.eye(256))))